# Adaptive PDE discretizations on cartesian grids 
## Volume : GPU accelerated methods
## Part : Reproducibility
## Chapter : Automatic differentiation

## 0. Importing the required libraries

In [1]:
import sys; sys.path.insert(0,"../..")
#from Miscellaneous import TocTools; print(TocTools.displayTOC('Isotropic_Repro','GPU'))

In [2]:
import cupy as cp
import numpy as np
import itertools
from copy import copy
from matplotlib import pyplot as plt
np.set_printoptions(edgeitems=30, linewidth=100000, formatter=dict(float=lambda x: "%5.3g" % x))

In [3]:
#np is np

In [4]:
from agd import AutomaticDifferentiation as ad
from agd import Metrics
from agd import FiniteDifferences as fd
from agd import LinearParallel as lp
import agd.AutomaticDifferentiation.cupy_generic as cugen
norm_infinity = ad.Optimization.norm_infinity

In [5]:
z=ad.Dense.identity(constant=cp.ones(2))

In [6]:
z+cp.ones(2)

denseAD(array([    2,     2]),
array([[    1,     0],
       [    0,     1]]))

In [7]:
z+z

denseAD(array([    2,     2]),
array([[    2,     0],
       [    0,     2]]))

In [5]:
#cp = ad.functional.decorate_module_functions(cp,cugen.set_output_dtype32) # Use float32 and int32 types in place of float64 and int64
#plt = ad.functional.decorate_module_functions(plt,cugen.cupy_get_args)

In [6]:
x_numpy = ad.Dense.identity((2,))
x_numpy.cupy_based()

[    0     0] [[    1     0]
 [    0     1]]


False

In [7]:
ad.Dense.identity((2,))

[    0     0] [[    1     0]
 [    0     1]]


denseAD(array([    0,     0]),
array([[    1,     0],
       [    0,     1]]))

In [7]:
denseAD_cupy = cugen.cupy_rebase(ad.Dense.denseAD)

In [8]:
denseAD_cupy.__bases__

(cupy.core.core.ndarray,)

In [10]:
x=cp.ones(2)
z = denseAD_cupy(x)
type(z.coef)
z,z.shape

__new__ [    1     1] ()


(denseAD(array([    1,     1]),array([], dtype=float64)), (2,))

In [11]:
z.cupy_based()

True

In [12]:
z.new(z.value,z.coef)

denseAD(array([    1,     1]),array([], dtype=float64))

In [13]:
type(z)

agd.AutomaticDifferentiation.Dense.denseAD_cupy

[    1     1] [[    1     0]
 [    0     1]]


AssertionError: 

## 1. Dense  

In [39]:
help(cp.ndarray)

Help on class ndarray in module cupy.core.core:

class ndarray(builtins.object)
 |  ndarray(shape, dtype=float, memptr=None, strides=None, order=u'C')
 |  Multi-dimensional array on a CUDA device.
 |  
 |      This class implements a subset of methods of :class:`numpy.ndarray`.
 |      The difference is that this class allocates the array content on the
 |      current GPU device.
 |  
 |      Args:
 |          shape (tuple of ints): Length of axes.
 |          dtype: Data type. It must be an argument of :class:`numpy.dtype`.
 |          memptr (cupy.cuda.MemoryPointer): Pointer to the array content head.
 |          strides (tuple of ints or None): Strides of data in memory.
 |          order ({'C', 'F'}): Row-major (C-style) or column-major
 |              (Fortran-style) order.
 |  
 |      Attributes:
 |          base (None or cupy.ndarray): Base array from which this array is
 |              created as a view.
 |          data (cupy.cuda.MemoryPointer): Pointer to the array conten

In [100]:
help(cp.ndarray.__new__)

Help on built-in function __new__:

__new__(*args, **kwargs) method of builtins.type instance
    Create and return a new object.  See help(type) for accurate signature.



In [76]:
y = cp.ndarray(shape=x.shape,dtype=x.dtype,
               memptr=x.data,
               strides=x.strides,order='C')

In [121]:
xp=cp
class myclass(xp.ndarray):
    def __new__(cls,x):
        obj = super(myclass,cls).__new__(cls,shape=2)
        #,dtype=x.dtype,memptr=x.data,strides=x.strides,order='C')
        obj.coef=2
        print(obj,type(obj))
        return obj
    def __init__(self,*args,**kwargs):
        print('args',args)
        print('kwargs',kwargs)
#    def __array_finalize__
#        obj=value.view(type=myclass)
#        return obj
z = myclass(x)
#z.coef=2
print(z,z.coef)

1.0 <class '__main__.myclass'>
args (array([    1,     1]),)
kwargs {}
1.0 2


In [111]:
xp=cp
class myclass(xp.ndarray):
    def __new__(cls,x):
        obj = super().__new__(myclass,shape=x.size,dtype=x.dtype,memptr=x.data,strides=x.strides,order='C')
        obj.coef=2
        return obj
    def __init__(self,*args,**kwargs):
        print('args',args)
        print('kwargs',kwargs)
#    def __array_finalize__
#        obj=value.view(type=myclass)
#        return obj
z = myclass(x)
#z.coef=2
print(z,z.coef)

args (array([    1,     1]),)
kwargs {}
1.0 2


In [112]:
z

array(    1)

In [85]:
z.coef

2

In [139]:
# This one works
xp=cp
class myclass2(xp.ndarray):
    def __new__(cls,**kwargs):
#        print(arr.size)
        obj = super(myclass2,cls).__new__(cls,**kwargs)
        obj.coef=2
        return obj

    @classmethod
    def cast(cls,x):
        return cls(shape=x.shape,dtype=x.dtype,
               memptr=x.data,
               strides=x.strides,order='C')
        
    
#    def __array_finalize__
#        obj=value.view(type=myclass)
#        return obj
z = myclass2.cast(x)
z,z.coef

(array([    1,     1]), 2)

In [185]:
xp=cp
def cupy_init_kwargs(x):
    return {
            'shape':x.shape,'dtype':x.dtype,
               'memptr':x.data,'strides':x.strides,'order':'C'
        }

class myclass5(xp.ndarray):    
    def __new__(cls,x):
        obj = super(myclass5,cls).__new__(cls,**cupy_init_kwargs(x))
        obj.coef=2
        return obj
    def __init__(self,x):
        super(myclass5,self).__init__(**cupy_init_kwargs(x))
        
    def value(self):
        return 
x=xp.ones(2)
z = myclass5(x)
print(z,z.coef,cp.asarray(z))
print(type(z),type(z.view()))

[    1     1] 2 [    1     1]
<class '__main__.myclass5'> <class 'cupy.core.core.ndarray'>


In [186]:
x.dtype

dtype('float64')

In [172]:
dir(xp.ndarray)

['T',
 '__abs__',
 '__add__',
 '__and__',
 '__array__',
 '__array_function__',
 '__array_priority__',
 '__array_ufunc__',
 '__bool__',
 '__class__',
 '__complex__',
 '__copy__',
 '__cuda_array_interface__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__divmod__',
 '__doc__',
 '__eq__',
 '__float__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__ifloordiv__',
 '__ilshift__',
 '__imod__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__irshift__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__lshift__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__or__',
 '__pos__',
 '__pow__',
 '__pyx_vtable__',
 '__radd__',
 '__rand__',
 '__rdivmod__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rfloordiv__',
 '__rlshift__',
 '__rmatmul__',
 '__rmod__',
 '_

In [159]:
myclass5.__bases__[0]

cupy.core.core.ndarray

In [161]:
type(z).__bases__

(cupy.core.core.ndarray,)

In [165]:
x0=cp.ones(2)
x1=cp.ascontiguousarray(x0)
x0 is x1

True

In [144]:
xp=cp
class myclass4(xp.ndarray):
    def __init__(self,x):
        super(myclass4,self).__init__(shape=x.shape,dtype=x.dtype,
               memptr=x.data,strides=x.strides,order='C')
        x.coef=2
z=myclass4(x)
z,z.coef

AttributeError: 'cupy.core.core.ndarray' object has no attribute 'coef'

In [134]:
def cupy_type_view(x,subclass):
    return super(cp.ndarray,subclass).__new__(subclass,shape=2)
    

In [135]:
class myclass3(cp.ndarray):
    pass
cupy_type_view(x,myclass3)

TypeError: object.__new__(myclass3) is not safe, use myclass3.__new__()

In [127]:
z,z.coef

(array([    0,     0]), 2)

In [81]:
x.shape

(2,)

In [82]:
z = myclass(x)

TypeError: cupy.core.core.ndarray.__new__(): not enough arguments

In [75]:
z is x

True

In [53]:
x=cp.ones(2)
x.__cuda_array_interface__

{'shape': (2,),
 'typestr': '<f8',
 'descr': [('', '<f8')],
 'data': (25874663424, False),
 'version': 0}

In [43]:
x._c_contiguous

True

In [56]:
x.data

In [57]:
y = cp.ndarray(shape=x.shape,dtype=x.dtype,
               memptr=x.data,
               strides=x.strides,order='C')

In [58]:
y

array([    1,     1])

In [50]:
help(cp.cuda.memory.BaseMemory)

Help on class BaseMemory in module cupy.cuda.memory:

class BaseMemory(builtins.object)
 |  Memory on a CUDA device.
 |  
 |  Attributes:
 |      ~Memory.ptr (int): Pointer to the place within the buffer.
 |      ~Memory.size (int): Size of the memory allocation in bytes.
 |      ~Memory.device (~cupy.cuda.Device): Device whose memory the pointer
 |          refers to.
 |  
 |  Methods defined here:
 |  
 |  __int__(...)
 |      Returns the pointer value to the head of the allocation.
 |  
 |  __reduce__ = __reduce_cython__(...)
 |      BaseMemory.__reduce_cython__(self)
 |  
 |  __setstate__ = __setstate_cython__(...)
 |      BaseMemory.__setstate_cython__(self, __pyx_state)
 |  
 |  ----------------------------------------------------------------------
 |  Static methods defined here:
 |  
 |  __new__(*args, **kwargs) from builtins.type
 |      Create and return a new object.  See help(type) for accurate signature.
 |  
 |  ------------------------------------------------------------

In [35]:
dir(cp.ones(2))

['T',
 '__abs__',
 '__add__',
 '__and__',
 '__array__',
 '__array_function__',
 '__array_priority__',
 '__array_ufunc__',
 '__bool__',
 '__class__',
 '__complex__',
 '__copy__',
 '__cuda_array_interface__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__divmod__',
 '__doc__',
 '__eq__',
 '__float__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__ifloordiv__',
 '__ilshift__',
 '__imod__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__irshift__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__lshift__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__or__',
 '__pos__',
 '__pow__',
 '__pyx_vtable__',
 '__radd__',
 '__rand__',
 '__rdivmod__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rfloordiv__',
 '__rlshift__',
 '__rmatmul__',
 '__rmod__',
 '_

In [10]:
z

array([None, None], dtype=object)

In [13]:
x = cp.zeros(2)
help(x.view)

Help on built-in function view:

view(...) method of cupy.core.core.ndarray instance
    ndarray.view(self, dtype=None) -> ndarray
    Returns a view of the array.
    
            Args:
                dtype: If this is different from the data type of the array, the
                    returned view reinterpret the memory sequence as an array of
                    this type.
    
            Returns:
                cupy.ndarray: A view of the array. A reference to the original
                array is stored at the :attr:`~ndarray.base` attribute.
    
            .. seealso:: :meth:`numpy.ndarray.view`



In [14]:
help(x.__new__)

Help on built-in function __new__:

__new__(*args, **kwargs) method of builtins.type instance
    Create and return a new object.  See help(type) for accurate signature.



In [21]:
x.order

AttributeError: 'cupy.core.core.ndarray' object has no attribute 'order'

In [15]:
help(cp.ndarray)

Help on class ndarray in module cupy.core.core:

class ndarray(builtins.object)
 |  ndarray(shape, dtype=float, memptr=None, strides=None, order=u'C')
 |  Multi-dimensional array on a CUDA device.
 |  
 |      This class implements a subset of methods of :class:`numpy.ndarray`.
 |      The difference is that this class allocates the array content on the
 |      current GPU device.
 |  
 |      Args:
 |          shape (tuple of ints): Length of axes.
 |          dtype: Data type. It must be an argument of :class:`numpy.dtype`.
 |          memptr (cupy.cuda.MemoryPointer): Pointer to the array content head.
 |          strides (tuple of ints or None): Strides of data in memory.
 |          order ({'C', 'F'}): Row-major (C-style) or column-major
 |              (Fortran-style) order.
 |  
 |      Attributes:
 |          base (None or cupy.ndarray): Base array from which this array is
 |              created as a view.
 |          data (cupy.cuda.MemoryPointer): Pointer to the array conten

In [5]:
ad.Dense.denseAD(cp.ones(2))

AttributeError: 'cupy.core.core.ndarray' object has no attribute 'coef'

In [5]:
cupy_denseAD = type('cupy_denseAD',(cp.ndarray,),dict(ad.Dense.denseAD.__dict__))
cupy_denseAD(cp.ones((2,)))

AttributeError: 'cupy.core.core.ndarray' object has no attribute 'coef'

In [8]:
x = np.ones(2)
x.coef=2

AttributeError: 'numpy.ndarray' object has no attribute 'coef'

denseAD(array([    1,     1]),array([], shape=(2, 0), dtype=float64))

In [6]:
{np.ndarray:1,cp.ndarray:2}

{numpy.ndarray: 1, cupy.core.core.ndarray: 2}

In [27]:
#help(type)

In [15]:
np_cls = ad.Dense.denseAD
cp_cls = copy(np_cls)
cp_cls.__bases__ = (cp.ndarray,)


TypeError: __bases__ assignment: 'cupy.core.core.ndarray' deallocator differs from 'numpy.ndarray'

ValueError: object __array__ method not producing an array

In [23]:
x=cupy_denseAD(cp.ones((2,)))

TypeError: 'cupy.core.core.ndarray' object cannot be interpreted as an integer

In [7]:
x = ad.Dense.identity((2,2))
#x.view(np.ndarray)

In [8]:
x = ad.Dense.identity((2,2))
super(type(x),x)

<super: agd.AutomaticDifferentiation.Dense.denseAD,
        denseAD(array([[    0,     0],
               [    0,     0]]),
        array([[[    1,     0,     0,     0],
                [    0,     1,     0,     0]],
        
               [[    0,     0,     1,     0],
                [    0,     0,     0,     1]]]))>

In [5]:
X = cp.arange(6).reshape(2,3)
X_ad = ad.Dense.identity(constant=X)

ValueError: object __array__ method not producing an array

In [14]:
a = np.zeros((2,2))
b = np.array(a)

In [15]:
b[0,0]=1
a,b

(array([[    0,     0],
        [    0,     0]]),
 array([[    1,     0],
        [    0,     0]]))

In [8]:
np.array(cp.arange(2))

ValueError: object __array__ method not producing an array